# 텍스트 분할(Text Splitter)
- 문서분할은 Retrieval-Augmented Generation(RAG) 시스템의 두 번째 단계로서, 로드된 문서들을 효율적으로 처리하고, 시스템이 정보를 보다 잘 활용할 수 있도록 준비하는 중요한 과정

    **크고 복잡한 문서를 LLM 이 받아들일 수 있는 효율적인 작은 규모의 조각으로 나누는 작업이며, 나중에 사용자가 입력한 질문에 대하여 보다 효율적인 정보만 압축/선별하여 가져오기 위함**

<br>

### 분할의 필요성
- **핀포인트 정보 검색(정확성)**: 문서를 세분화함으로써 질문(Query) 에 연관성이 있는 정보만 가져오는데 도움
  - 각각의 단위는 특정 주제나 내용에 초점을 맞추므로, 관련성이 높은 정보를 제공
- **리소스 최적화(효율성)**: 전체 문서를 LLM 으로 입력하게 되면 비용이 많이 발생할 뿐더러, 효율적인 답변을 많은 정보속에 발췌하여 답변하지 못하게 됨
  - 이러한 문제가 할루시네이션으로 이어지게 되며, 답변에 필요한 정보만 발췌하기 위한 목적

<br>

### 문서분할 과정
1. **문서 구조 파악**: PDF 파일, 웹 페이지, 전자 책 등 다양한 형식의 문서에서 구조를 파악. 이는 문서의 헤더, 푸터, 페이지 번호, 섹션 제목 등을 식별하는 과정을 포함
2. **단위 선정**: 문서를 어떤 단위로 나눌지 결정. 이는 페이지별, 섹션별, 또는 문단별일 수 있으며, 문서의 내용과 목적에 따라 다름
3. **단위 크기 선정(chunk size)**: 문서를 몇 개의 토큰 단위로 나눌 것인지를 결정
4. **청크 오버랩(chunk overlap)**: 분할된 끝 부분에서 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할하는 것이 일반적

<br>

<hr>

<br>

## 문자 텍스트 분할(`CharacterTextSplitter`)

<br>

### `CharacterTextSplitter`
- 가장 간단한 방식
- `"\n\n"` 을 기준으로 문자 단위로 텍스트를 분할하고, 청크의 크기를 문자 수로 측정
    - **텍스트 분할 방식**: **단일 문자** 기준
    - **청크 크기 측정 방식**: **문자 수** 기준

In [1]:
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

<br>

#### 텍스트를 청크(chunk)로 분할
- `separator` : 매개변수로 분할할 기준을 설정. 기본 값은 `"\n\n"`
- `chunk_size` : 각 청크의 최대 크기를 제한
- `chunk_overlap` : 인접한 청크 간에 몇 자의 중복을 허용할지 설정
- `length_function` : 텍스트의 길이를 계산하는 함수를 지정
- `is_separator_regex` : `separator`를 정규식이 아닌 일반 문자열로 처리 $\rightarrow$ `False`

<br>



In [4]:
from langchain_text_splitters import CharacterTextSplitter

In [6]:
text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자를 지정. 기본값은 "\n\n"
    separator="\n\n",
    
    # 분할된 텍스트 청크의 최대 크기를 지정
    chunk_size=250,
    
    # 분할된 텍스트 청크 간의 중복되는 문자 수를 지정
    chunk_overlap=50,
    
    # 텍스트의 길이를 계산하는 함수를 지정
    length_function=len,
    
    # 구분자가 정규식인지 여부를 지정
    is_separator_regex=False,
)

- file 텍스트를 문서 단위로 분할

In [9]:
texts = text_splitter.create_documents([file])
print(texts[0])

page_content='Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding'


<br>

#### 메타데이터가 문서와 함께 분할
- `create_documents` 메서드는 텍스트 데이터와 메타데이터 리스트를 인자롭 받음

In [12]:
metadatas = [
    {"document": 1},
    {"document": 2},
]  # 문서에 대한 메타데이터 리스트를 정의

documents = text_splitter.create_documents(
    [
        file,
        file,
    ],  # 분할할 텍스트 데이터를 리스트로 전달
    metadatas=metadatas,  # 각 문서에 해당하는 메타데이터를 전달
)
print(documents[0])

page_content='Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding' metadata={'document': 1}


In [13]:
text_splitter.split_text(file)[0]

'Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding'

<br>

<hr>

<br>

## 재귀적 문자 텍스트 분할(`RecursiveCharacterTextSplitter`)
- **일반적인 텍스트에 권장되는 방식**
- **문자 목록을 매개변수로 받아 동작 $\rightarrow$ 분할기는 청크가 충분히 작아질 때까지 주어진 문자 목록의 순서대로 텍스트를 분할하려고 시도**
  - 기본 문자 목록은 `["\n\n", "\n", " ", ""]`

<br>

- **단락 $\rightarrow$ 문장 $\rightarrow$ 단어 순서로 재귀적으로 분할**
  - 단락(그 다음으로 문장, 단어) 단위가 의미적으로 가장 강하게 연관된 텍스트 조각으로 간주되므로, 가능한 한 함께 유지하려는 효과
  - **텍스트가 분할되는 방식**: 문자 목록(`["\n\n", "\n", " ", ""]`) 에 의해 분할
  - **청크 크기가 측정되는 방식**: 문자 수에 의해 측정

In [14]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

<br>

#### `RecursiveCharacterTextSplitter`
- `chunk_size` : 250 으로 설정하여 각 청크의 크기를 제한
- `chunk_overlap` : 50 으로 설정하여 인접한 청크 간에 50 개 문자의 중첩을 허용
- `length_function` : len 함수를 사용하여 텍스트의 길이를 계산
- `is_separator_regex` : `False`로 설정하여 구분자로 정규식을 사용하지 않음

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [18]:
texts = text_splitter.create_documents([file])

print(texts[0])
print("===" * 20)
print(texts[1])

page_content='Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding'
page_content='Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token'


In [ ]:
text_splitter.split_text(file)[:2]  

['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding',
 'Embedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken']

<br>

<hr>

<br>

## 토큰 텍스트 분할(`TokenTextSplitter`)
- 언어 모델에는 토큰 제한 $\rightarrow$ 토큰 제한을 초과하지 않아야 함
  
  `TokenTextSplitter` 는 텍스트를 토큰 수를 기반으로 청크를 생성할 때 유용

<br>

### `tiktoken`

In [20]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

In [25]:
print(file[:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어


<br>

- `from_tiktoken_encoder` 메서드를 사용하여 `Tiktoken` 인코더 기반의 텍스트 분할기를 초기화
  - `CharacterTextSplitter.from_tiktoken_encoder`를 사용할 경우, 텍스트는 `CharacterTextSplitter`에 의해서만 분할되고 `tiktoken` 토크나이저는 분할된 텍스트를 병합하는 데 사용
    
    $\rightarrow$ **분할된 텍스트가 `tiktoken` 토크나이저로 측정한 청크 크기보다 클 수 있음**

  - `RecursiveCharacterTextSplitter.from_tiktoken_encoder`를 사용할 경우, 분할된 텍스트가 언어 모델에서 허용하는 토큰의 청크 크기보다 크지 않도록 할 수 있으며, 각 분할은 크기가 더 큰 경우 재귀적으로 분할
    
    $\rightarrow$ **각 분할이 청크 크기보다 작음을 보장**

In [21]:
from langchain_text_splitters import CharacterTextSplitter

In [26]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    # 청크 크기를 300으로 설정
    chunk_size=300,
    
    # 청크 간 중복되는 부분이 없도록 설정
    chunk_overlap=0,
)

- file 텍스트를 청크 단위로 분할

In [27]:
texts = text_splitter.split_text(file)

Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 315, which is longer than the specified 300
Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 366, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 351, which is longer than the specified 300
Created a chunk of size 378, which is longer than the specified 300
Created a chunk of size 361, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 335, which is longer than the specified 300
Created a chunk of size 353, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 336, which is longer than the specified 300
Created a chunk of size 324, which is longer tha

In [ ]:
print(len(texts)) # 분할된 청크의 개수
print(texts[0]) # texts 리스트의 첫 번째 요소를 출력

51
Semantic Search


<br>

### `TokenTextSplitter`

In [30]:
from langchain_text_splitters import TokenTextSplitter

In [31]:
text_splitter = TokenTextSplitter(
    chunk_size=200,
    chunk_overlap=0,  # 청크 간 중복을 0으로 설정
)

In [33]:
texts = text_splitter.split_text(file)
print(texts[0])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"�


<br>

### `spaCy`

```bash
$ pip install --upgrade --quiet spacy
$ python -m spacy download en_core_web_sm --quiet
```

In [36]:
!python -m spacy download en_core_web_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

In [6]:
from langchain_text_splitters import SpacyTextSplitter
warnings.filterwarnings("ignore")

In [4]:
text_splitter = SpacyTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)

In [8]:
texts = text_splitter.split_text(file)
print(texts[0], '\n')
print(texts[1])    

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된

결과를 반환하는 검색 방식입니다.


예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 

연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나

문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다.

이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.


예시: "사과"라는 단어를

[0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.


<br>

### NLTK    

- 단순히 `"\n\n"`으로 분할하는 대신, NLTK tokenizers를 기반으로 텍스트를 분할

```bash
$ pip install -qU nltk
```

In [16]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

In [ ]:
from langchain_text_splitters import NLTKTextSplitter

In [19]:
text_splitter = NLTKTextSplitter(
    chunk_size=200,
    chunk_overlap=0,
)

In [20]:
texts = text_splitter.split_text(file)
print(texts[0])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.

예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


<br>

<hr>

<br>

## 시멘틱 청커(SemanticChunker)
* `SemanticChunker`는 LangChain의 실험적 기능 중 하나로, 텍스트를 의미론적으로 유사한 청크로 분할하는 역할


In [31]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

In [37]:
print(file[:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어


In [30]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

- OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화

In [32]:
text_splitter = SemanticChunker(OpenAIEmbeddings())

- 텍스트 분할

In [ ]:
chunks = text_splitter.split_text(file)
print(chunks[0][:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다. 예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다. 예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다. 연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다. 예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다. 연관키워드: 토큰화, 자연어


- 청크를 문서로 변환

In [39]:
docs = text_splitter.create_documents([file])
print(docs[0].page_content[:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다. 예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다. 예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다. 연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다. 예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다. 연관키워드: 토큰화, 자연어


<br>

### Breakpoints
- 문장을 "분리"할 시점을 결정하여 작동 **(두 문장 간의 임베딩 차이)**

    $\rightarrow$ 그 차이가 특정 임계값을 넘으면 문장이 분리

<br>

#### Percentile
- **기본적인 분리 방식은 백분위수(`Percentile`) 를 기반**
- 문장 간의 모든 차이를 계산한 다음, 지정한 백분위수를 기준으로 분리

In [41]:
text_splitter = SemanticChunker(
    
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화
    OpenAIEmbeddings(),
    
    # 분할 기준점 유형을 백분위수로 설정
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70,
)

In [ ]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)
    print("===" * 20, '\n')

[Chunk 0]

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다. 예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.

[Chunk 1]

예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다. 연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.

[Chunk 2]

예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.

[Chunk 3]

예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.

[Chunk 4]

예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다. 연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽

In [44]:
print(len(docs))

27


<br>

### Standard Deviation
- **지정한 `breakpoint_threshold_amount` 표준편차보다 큰 차이가 있는 경우 분할**

In [45]:
text_splitter = SemanticChunker(
    
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화
    OpenAIEmbeddings(),
    
    # 분할 기준으로 표준 편차를 사용
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=1.25,
)

In [51]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)
    print("===" * 20)

[Chunk 0]

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다. 예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
[Chunk 1]

예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다. 연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다. 예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
[Chunk 2]

예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
[Chunk 3]

예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다. 연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽입, 삭제 등 다양한 작업을

<br>

### Interquartile
- 사분위수 범위(interquartile range)를 사용하여 청크를 분할

In [48]:
text_splitter = SemanticChunker(
    
    # OpenAI의 임베딩 모델을 사용하여 의미론적 청크 분할기를 초기화
    OpenAIEmbeddings(),
    
    # 분할 기준점 임계값 유형을 사분위수 범위로 설정
    breakpoint_threshold_type="interquartile",
    breakpoint_threshold_amount=0.5,
)

In [49]:
docs = text_splitter.create_documents([file])

for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)
    print("===" * 20)

[Chunk 0]

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다. 예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
[Chunk 1]

예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다. 연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다. 예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
[Chunk 2]

예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
[Chunk 3]

예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다. 연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽입, 삭제 등 다양한 작업을

<br>

<hr>